# 신경망 모델 구성하기
##### 신경망은 데이터에 대한 연산을 수행하는 계층/모듈로 구성되어 있다.
##### torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공한다.
##### PyTorch의 모든 모듈은 nn.Module의 하위 클래스이다. 신경망은 다른 모듈로 구성된 모듈이다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있다.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 학습을 위한 장치 얻기
##### 가능한 경우 GPU 또는 MPS와 같은 하드웨어 가속기에서 모델을 학습하려고 한다.
##### torch.cuda 또는 torch.backends.mps가 사용 가능한지 확인해보고, 그렇지 않으면 CPU를 계속 사용한다.

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(device)

cpu


## 클래스 정의하기
##### 신경망 모델을 nn.Module의 하위클래스로 정의하고, _ _init_ _에서 신경망 계층들을 초기화한다. nn.Module을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현한다.

In [8]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

##### NeuralNetwork의 인스턴스를 생성하고 device 이동 후 구조 출력


In [9]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


##### 모델을 사용하기 위해 입력 데이터를 전달한다. 이는 일부 백그라운드 연산들과 함께 모델의 forward를 실행한다. (model.forward()를 직접 호출하지 마라!)
##### 모델에 입력을 전달하여 호출하면 2차원 텐서를 반환한다. 2차원 텐서의 dim=0은 각 분류에 대한 원시 예측값 10개가, dim=1에는 각 출력의 개별 값들이 해당한다.
##### 원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻는다.

In [10]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(y_pred)

tensor([1])


## 모델 계층(Layer)
##### FashionMNIST 모델의 계층들을 살펴보자. 임시로 28 X 28 크기의 이미지 3개로 구성된 미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 볼 것이다.

In [11]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten
##### nn.Flatten : 계층을 초기화하여 각 28 x 28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 반환한다.

In [12]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear
##### 선형 계층 : 저장된 가중치와 편향을 사용하여 입력에 선형 변환을 적용하는 모듈이다.

In [13]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU
##### 비선형 활성화는 모델의 입력과 출력 사이에 복잡한 관계를 만든다. 비선형 활성화는 선형 변환 후에 적용되어 비선형성을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕는다.
##### 이 모델에서는 nn.ReLU를 선형 계층들 사이에 사용하지만, 모델을 만들 때는 비선형성을 가진 다른 활성화를 도입할 수도 있다.

In [14]:
print(hidden1)
hidden1 = nn.ReLU()(hidden1)
print(hidden1)

tensor([[-0.2956,  0.3624,  0.1105, -0.3767,  0.0900, -0.1504, -0.0308, -0.2705,
          0.5809,  0.1460, -0.1946, -0.2400, -0.3889, -0.5061,  0.1346,  0.1025,
         -0.5879, -0.1551, -0.0907,  0.0187],
        [-0.3058,  0.2798,  0.1210, -0.3111,  0.2904,  0.0845, -0.0598, -0.6058,
          0.5334,  0.0027, -0.2465, -0.1340, -0.3277, -0.1591, -0.0327,  0.1774,
         -0.0294, -0.2157, -0.1473, -0.1175],
        [-0.2186,  0.4247,  0.2552, -0.2951,  0.0058,  0.2079, -0.1781, -0.1836,
          0.6106,  0.2582, -0.3131,  0.0051, -0.5280, -0.2735,  0.0240, -0.1589,
         -0.2537, -0.2357, -0.1165, -0.3570]], grad_fn=<AddmmBackward0>)
tensor([[0.0000, 0.3624, 0.1105, 0.0000, 0.0900, 0.0000, 0.0000, 0.0000, 0.5809,
         0.1460, 0.0000, 0.0000, 0.0000, 0.0000, 0.1346, 0.1025, 0.0000, 0.0000,
         0.0000, 0.0187],
        [0.0000, 0.2798, 0.1210, 0.0000, 0.2904, 0.0845, 0.0000, 0.0000, 0.5334,
         0.0027, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1774, 0.0000, 0.0000,

### nn.Sequential
##### nn.Sequential : 순서를 갖는 모듈의 컨테이너이다. 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달된다. 순차 컨테이너를 사용하여 아래의 seq_modules와 같은 신경망을 빠르게 만들 수 있다.

In [16]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
logits

tensor([[ 0.0985, -0.1045,  0.0437,  0.2492,  0.0974, -0.2177,  0.0735, -0.0645,
          0.2221, -0.2123],
        [ 0.0283, -0.0846,  0.0422,  0.2375,  0.0527, -0.2745,  0.0976, -0.0827,
          0.2205, -0.2061],
        [ 0.0740, -0.0555, -0.0360,  0.2252,  0.0765, -0.2351,  0.0689, -0.0713,
          0.2760, -0.0414]], grad_fn=<AddmmBackward0>)

### nn.Softmax
##### 신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 logits를 반환한다.
##### logits는 모델의 각 분류에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정된다. dim 매개변수는 값의 합이 1이 되는 차언을 나타낸다.

In [17]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [18]:
pred_probab

tensor([[0.1070, 0.0874, 0.1013, 0.1244, 0.1069, 0.0780, 0.1044, 0.0909, 0.1211,
         0.0784],
        [0.1013, 0.0905, 0.1027, 0.1249, 0.1038, 0.0748, 0.1086, 0.0906, 0.1227,
         0.0801],
        [0.1036, 0.0911, 0.0928, 0.1206, 0.1039, 0.0761, 0.1031, 0.0896, 0.1268,
         0.0923]], grad_fn=<SoftmaxBackward0>)

## 모델 매개변수
##### 신경망 내부의 많은 계층들은 매개변수화된다. 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어진다.
##### nn.Module을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적되며, 모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있다.

In [19]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0231, -0.0206, -0.0115,  ..., -0.0223, -0.0351,  0.0082],
        [ 0.0272, -0.0162, -0.0340,  ...,  0.0334, -0.0115,  0.0159]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0319, 0.0301], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0375, -0.0053, -0.0314,  ..., -0.0408, -0.0323, -0.0326],
        [-0.0158,  0.0019, -0.0406,  ..., -0.0019,  0.0052, -0.0337]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Si